In [1]:
!pip install -q openprompt==0.1.1 \
'torch>=1.9.0' \
'transformers>=4.10.0' \
sentencepiece==0.1.96 \
'scikit-learn>=0.24.2' \
'tqdm>=4.62.2' \
tensorboardX \
nltk \
yacs \
dill \
datasets \
rouge==1.0.0 \
scipy==1.4.1 \
fugashi \
ipadic \
unidic-lite

In [2]:
import openprompt.plms as plms
from openprompt.plms.mlm import MLMTokenizerWrapper
from transformers import BertConfig, BertForMaskedLM, BertTokenizer
from openprompt.data_utils import InputExample
from transformers import AutoTokenizer, AutoModel
from openprompt.plms import load_plm
from openprompt.prompts import ManualTemplate
from openprompt.prompts import PtuningTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt import PromptForClassification
from openprompt import PromptDataLoader
import torch



In [3]:
import xml.etree.ElementTree as ET
tree = ET.parse('./smokers_surrogate_train_all_version2.xml')
root = tree.getroot()

In [4]:
root.attrib
root.tag
counter=0
for child in root:
    if counter == 10:
        break
    print(child.tag, child.attrib)
    counter +=1
    

RECORD {'ID': '641'}
RECORD {'ID': '643'}
RECORD {'ID': '681'}
RECORD {'ID': '704'}
RECORD {'ID': '757'}
RECORD {'ID': '786'}
RECORD {'ID': '872'}
RECORD {'ID': '874'}
RECORD {'ID': '535'}
RECORD {'ID': '540'}


In [5]:
[elem.tag for elem in root.iter()]

counter=0
for elem in root.iter():
    if counter==10:
        break
    print(elem.tag)
    counter+=1

ROOT
RECORD
SMOKING
TEXT
RECORD
SMOKING
TEXT
RECORD
SMOKING
TEXT


In [6]:
counter=0
for smoking in root.iter('SMOKING'):
    if counter==10:
        break
    print(smoking.attrib.get('STATUS'))
    counter +=1
    
# for movie in root.iter('SMOKING'):
#     print(movie.attrib)

CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER
CURRENT SMOKER


In [7]:
# for text in root.iter('TEXT'):
#     print(text.text)
    
counter=0
for text in root.iter('TEXT'):
    if counter==1:
        break
    print(text.text)
    counter +=1    


977146916
HLGMC
2878891
022690
01/27/1997 12:00:00 AM
CARCINOMA OF THE COLON .
Unsigned
DIS
Report Status :
Unsigned
Please do not go above this box important format codes are contained .
DISCHARGE SUMMARY
ARF32 FA
DISCHARGE SUMMARY NAME :
GIRRESNET , DIEDREO A
UNIT NUMBER :
075-71-01
ADMISSION DATE :
01/27/1997
DISCHARGE DATE :
01/31/1997
PRINCIPAL DIAGNOSIS :
Carcinoma of the colon .
ASSOCIATED DIAGNOSIS :
Urinary tract infection , and cirrhosis of the liver .
HISTORY OF PRESENT ILLNESS :
The patient is an 80-year-old male , who had a history of colon cancer in the past , resected approximately ten years prior to admission , history of heavy alcohol use , who presented with a two week history of poor PO intake , weight loss , and was noted to have acute on chronic Hepatitis by chemistries and question of pyelonephritis .
He lived alone but was driven to the hospital by his son because of reported worsening and general care and deconditioning .
Emergency Department course ; he was ev

In [8]:
counter=0
for text in root.findall("./SMOKING/[@STATUS='CURRENT SMOKER']"):
    if counter==10:
        break
    print(text.attrib)
    print(counter)
    counter +=1    

In [9]:
# doc=xee.fromstring('./smokers_surrogate_train_all_version2.xml')

for tag in root.findall('SMOKING'):
    if tag.attrib['STATUS']!='CURRENT SMOKER':
#         root.remove(tag)
        print(tag)
        
# print(ET.tostring(root))

In [10]:
# node=root
# if node.tag == "SMOKING":
#         status = node.attrib.get("STATUS")
#         print(status)
#         if status is not None:
#             print(node.attrib.get("TEXT"))
import xml.etree.ElementTree as ET
tree = ET.parse('./smokers_surrogate_train_all_version2.xml')
root = tree.getroot()

text_list=[]
counter =0             
for record in root.iter('RECORD'):
    if counter == 2:
        break
    print(record.attrib.get('ID'))
    smoking = record.find('SMOKING')
    print(smoking.attrib.get('STATUS'))
    if smoking.attrib.get('STATUS')=="CURRENT SMOKER":
        text=record.find('TEXT')
        text_list.append(text)
        print(counter)
    counter +=1

641
CURRENT SMOKER
0
643
CURRENT SMOKER
1


In [11]:
import xml.etree.ElementTree as ET
tree = ET.parse('./smokers_surrogate_train_all_version2.xml')
root = tree.getroot()

text_list=[]
label_list=[]
counter =0             
for record in root.iter('RECORD'):
#     if counter:
#         break
#     print(record.attrib.get('ID'))
    smoking = record.find('SMOKING')
    label_list.append(smoking.attrib.get('STATUS'))
#     print(smoking.attrib.get('STATUS'))
    text=record.find('TEXT')
    text_list.append(text.text)
#     print(text_list.__len__())
#     print(counter)
    counter +=1

In [12]:
print(text_list.__len__())
print(label_list.__len__())

398
398


In [13]:
text_list.__class__
label_list.__class__

list

In [14]:
text_list[1]

'\n026738007\nCMC\n15319689\n3/25/1998 12:00:00 AM\nMEDIASTINAL ADENOPATHY RT. LUNG MASS\nSigned\nDIS\nAdmission Date :\n03/25/1998\nReport Status :\nSigned\nDischarge Date :\n04/08/1998\nPRINCIPAL DIAGNOSIS :\nRight lung choriocarcinoma .\nOTHER DIAGNOSIS :\nPericardial effusion .\nHISTORY OF PRESENT ILLNESS :\nThe patient was in her usual state of health , carrying the diagnosis of right lung chorio carcinoma , when she began to develop dyspnea and on 03/24/98 an echocardiogram revealed a pericardial effusion and possible tamponade clinically .\nThe patient was first diagnosed in 10/97 , when she presented with the symptoms of menorrhagia .\nShe had elevated beta HCG levels , no tubal pregnancy was suspected .\nAright oophorectomy was performed .\nIn 11/97 , chest x-ray revealed an abnormality in the right upper lobe .\nA lung biopsy was performed , revealing chorio carcinoma pathologically .\nAt the time , she also had symptoms of hemoptysis .\nOn November 26 , a chest CT revealed m

# Datset

In [15]:
import torch
from torch.utils.data.dataset import Dataset
import pandas as pd
from torch.utils.data import Dataset, DataLoader
 
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)
 
 
class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
        
    def __len__(self):
            return len(self.labels)
        
    def __getitem__(self, idx):
            label = self.labels[idx]
            text = self.text[idx]
            sample = {"Text": text, "Class": label}
            return sample

cpu


In [16]:
import pandas as pd

# text = ['Happy', 'Amazing', 'Sad', 'Unhapy', 'Glum']
# labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']

text_labels_df = pd.DataFrame({'Text': text_list, 'Labels': label_list})

TD = CustomTextDataset(text_labels_df['Text'],
                       text_labels_df['Labels'])

In [17]:
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(TD)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(TD), '\n')
# Print entire data set
# print('Entire data set: ', list(DataLoader(TD)), '\n')


First iteration of data set:  {'Text': "\n977146916\nHLGMC\n2878891\n022690\n01/27/1997 12:00:00 AM\nCARCINOMA OF THE COLON .\nUnsigned\nDIS\nReport Status :\nUnsigned\nPlease do not go above this box important format codes are contained .\nDISCHARGE SUMMARY\nARF32 FA\nDISCHARGE SUMMARY NAME :\nGIRRESNET , DIEDREO A\nUNIT NUMBER :\n075-71-01\nADMISSION DATE :\n01/27/1997\nDISCHARGE DATE :\n01/31/1997\nPRINCIPAL DIAGNOSIS :\nCarcinoma of the colon .\nASSOCIATED DIAGNOSIS :\nUrinary tract infection , and cirrhosis of the liver .\nHISTORY OF PRESENT ILLNESS :\nThe patient is an 80-year-old male , who had a history of colon cancer in the past , resected approximately ten years prior to admission , history of heavy alcohol use , who presented with a two week history of poor PO intake , weight loss , and was noted to have acute on chronic Hepatitis by chemistries and question of pyelonephritis .\nHe lived alone but was driven to the hospital by his son because of reported worsening and gene

# Dataset pre-processing

In [18]:
text_labels_df=text_labels_df[~text_labels_df.Labels.str.contains("UNKNOWN")]

In [19]:
text_labels_df_nonsmoker =text_labels_df[text_labels_df.Labels.str.contains("NON-SMOKER")]
text_labels_df_nonsmoker.head()

,Text,Labels
17,\n139964603\nNVH\n40876531\n11/17/2003 12:00:0...,NON-SMOKER
18,\n910458031\nNVH\n23238893\n1/12/1991 12:00:00...,NON-SMOKER
19,\n991509188\nCTMC\n51356028\n369929\n09/05/199...,NON-SMOKER
20,\n320422564\nCTMC\n09300549\n340424\n06/16/199...,NON-SMOKER
21,\n988183518\nPUOMC\n7961198\n290757\n137082\n1...,NON-SMOKER


In [20]:
text_labels_df_smoker =text_labels_df[~text_labels_df.Labels.str.contains("NON-SMOKER")]
text_labels_df_smoker.Labels = "SMOKER"
text_labels_df_smoker.head()

/ihome/yanshanwang/sos86/.local/lib/python3.8/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Text,Labels
0,\n977146916\nHLGMC\n2878891\n022690\n01/27/199...,SMOKER
1,\n026738007\nCMC\n15319689\n3/25/1998 12:00:00...,SMOKER
2,\n071962960\nBH\n4236518\n417454\n12/10/2001 1...,SMOKER
3,\n418520250\nNVH\n61562872\n3/11/1995 12:00:00...,SMOKER
4,\n301443520\nCTMC\n49020928\n448922\n1/11/1990...,SMOKER


In [21]:
text_labels_df_smoker.Labels.value_counts()

SMOKER    80
Name: Labels, dtype: int64

In [22]:
df = text_labels_df_smoker.append(text_labels_df_nonsmoker, ignore_index=True)

In [23]:
df.Labels.value_counts()

SMOKER        80
NON-SMOKER    66
Name: Labels, dtype: int64

In [24]:
x_test=df.Text.to_list()
y_test=df.Labels.to_list()
x_test[1].__class__

str

# BERT
OpenPrompt

In [25]:
import openprompt.plms as plms
from openprompt.plms.mlm import MLMTokenizerWrapper
from transformers import BertConfig, BertForMaskedLM, BertTokenizer

In [26]:
plms._MODEL_CLASSES['bert'] = plms.ModelClass(**{
    'config': BertConfig,
    'tokenizer': BertTokenizer,
    'model':BertForMaskedLM,
    'wrapper': MLMTokenizerWrapper,
})

In [27]:
plms._MODEL_CLASSES

{'bert': ModelClass(config=<class 'transformers.models.bert.configuration_bert.BertConfig'>, tokenizer=<class 'transformers.models.bert.tokenization_bert.BertTokenizer'>, model=<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>, wrapper=<class 'openprompt.plms.mlm.MLMTokenizerWrapper'>),
 'roberta': ModelClass(config=<class 'transformers.models.roberta.configuration_roberta.RobertaConfig'>, tokenizer=<class 'transformers.models.roberta.tokenization_roberta.RobertaTokenizer'>, model=<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>, wrapper=<class 'openprompt.plms.mlm.MLMTokenizerWrapper'>),
 'albert': ModelClass(config=<class 'transformers.models.albert.configuration_albert.AlbertConfig'>, tokenizer=<class 'transformers.models.albert.tokenization_albert.AlbertTokenizer'>, model=<class 'transformers.models.albert.modeling_albert.AlbertForMaskedLM'>, wrapper=<class 'openprompt.plms.mlm.MLMTokenizerWrapper'>),
 'gpt': ModelClass(config=<class 'transfo

# Step 1: Define a task
|

In [28]:

def prompt_classification(text):
    classes = [ 
        "SMOKER",
        "NON-SMOKER",
    ]
#     print(text)

    dataset = [ 
          InputExample(
    #         guid = 0,
            text_a =text
#               "Asthma affects lungs  and can be hard to diagnose. The signs of asthma can seem like the signs of COPD, pneumonia, bronchitis, pulmonary embolism, anxiety, and heart disease.", #lung
        ),
    #     InputExample(
    #         guid = 1,
    #         text_a = "COVID-19 is caused by a coronavirus called SARS-CoV-2", #virus
    #     ),
    #     InputExample(
    #         guid = 2,
    #         text_a = "When your brain is damaged, it can affect many different things, including your memory, your sensation, and even your personality. Brain disorders include any conditions or disabilities that affect your brain.", #brain
    #     ),
    #     InputExample(
    #         guid = 3,
    #         text_a = "Symptoms may appear 2-14 days after exposure to the virus", #virus
    #     ),
#             InputExample(
#     #         guid = 4,
#             text_a = """Neurodegenerative diseases cause your brain and nerves to deteriorate over time. They can change your personality and cause confusion. They can also destroy your brain’s tissue and nerves.

#     Some brain diseases, such as Alzheimer’s disease, may develop as you age. """, #brain
#         ),
    ]


#     plm, tokenizer, model_config, WrapperClass = load_plm_fn()


    # template_text = '{"placeholder":"text_a"}: This effects {"mask"}'
    template_text= 'A {"mask"} disorder :  {"placeholder": "text_a"}'

    promptTemplate = ManualTemplate(
        text = template_text,
        tokenizer = tokenizer,
    )

    promptVerbalizer = ManualVerbalizer(
        classes = classes,
        label_words = {
            "SMOKER": ["smoke", "smoking", "tobacco", "lung", "chest"],
            "NON-SMOKER": ["healthy"] #"healthy", 
        },
        tokenizer = tokenizer,
    )

    promptModel = PromptForClassification(
        template = promptTemplate,
        plm = plm,
        verbalizer = promptVerbalizer
    )


    data_loader = PromptDataLoader(
        dataset = dataset,
        tokenizer = tokenizer, 
        template = promptTemplate, 
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=256, decoder_max_length=3, 
        batch_size=1,shuffle=False, teacher_forcing=False, predict_eos_token=False,
        truncate_method="head"
    )

    promptModel.eval()
    with torch.no_grad():
        for batch in data_loader:
            logits = promptModel(batch)
    #         print(logits)
            preds = torch.argmax(logits, dim = -1)
#             print(classes[preds])
            
    return classes[preds]




In [29]:
#if __name__ == "__main__":
    
#     st.title("HealthPrompt: Classifying clinical texts")
#     st.write("Upload Clinical text, Classify it..")

#     @st.cache(allow_output_mutation=True)
def load_plm_fn():
        plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-uncased")
#         plm, tokenizer, model_config, WrapperClass = load_plm("bert", "emilyalsentzer/Bio_ClinicalBERT")
#         plm, tokenizer, model_config, WrapperClass = load_plm("bert", "mrm8488/bioclinicalBERT-finetuned-covid-papers")
        return plm, tokenizer, model_config, WrapperClass
    
#     with st.spinner("Loading PLM into memory..."):
plm, tokenizer, model_config, WrapperClass = load_plm_fn()
        
#     text = st.text_input('Enter the clinical text here: ')
        
#     if text:
#         st.write("Response: ")
#         with st.spinner("Searching for classes.."):
#             res=prompt_classification(text)
#             st.write("Class : {} - disease".format(res))
#         st.write("")

# res=prompt_classification(text)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
y_pred=[]
for x in x_test:
    pred_label=prompt_classification(x)
#     print(pred_label)
    y_pred.append(pred_label)
    
y_pred.__len__()

tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (961 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 1it [00:00, 61.08it/s]
tokenizing: 1it [00:00, 46.24it/s]
tokenizing: 1it [00:00, 33.35it/s]
tokenizing: 1it [00:00, 28.12it/s]
tokenizing: 1it [00:00, 81.70it/s]
tokenizing: 1it [00:00, 48.54it/s]
tokenizing: 1it [00:00, 73.16it/s]
tokenizing: 1it [00:00, 49.50it/s]
tokenizing: 1it [00:00, 53.84it/s]
tokenizing: 1it [00:00, 52.80it/s]
tokenizing: 1it [00:00, 78.01it/s]
tokenizing: 1it [00:00, 27.52it/s]
tokenizing: 1it [00:00, 28.18it/s]
tokenizing: 1it [00:00, 34.77it/s]
tokenizing: 1it [00:00, 48.35it/s]
tokenizing: 1it [00:00, 36.08it/s]
tokenizing: 1it [00:00, 109.63it/s]
tokenizing: 1it [00:00, 31.05it/s]
tokenizing: 1it [00:00, 38.62it/s]
tokenizing: 1it [00:00, 53.26it/s]
tokenizing: 1it [00:00, 88.22it/s]
tokenizing: 1it [00:00, 34.66it/s]
tokenizing: 1it [

146

In [31]:
# # making zero-shot inference using pretrained MLM with prompt
# import torch
# promptModel.eval()
# with torch.no_grad():
#     for batch in data_loader:
#         logits = promptModel(batch)
#         print(logits)
#         preds = torch.argmax(logits, dim = -1)
        
#         print(classes[preds])
# # predictions would be 1, 0 for classes 'positive', 'negative'


In [34]:
from sklearn.metrics import confusion_matrix
import numpy as np
confusion = confusion_matrix(y_test, y_pred)
print(confusion)
print("Classification accuracy is: ", (confusion[0][0] + confusion[1][1]) / np.sum(confusion))

[[ 0 66]
 [ 0 80]]
Classification accuracy is:  0.547945205479452


In [35]:
# DEBUGGING
df_debug = pd.DataFrame({'Text': x_test, 'Labels': y_test, 'Predicted':y_pred})
df_debug.head()

,Text,Labels,Predicted
0,\n977146916\nHLGMC\n2878891\n022690\n01/27/199...,SMOKER,SMOKER
1,\n026738007\nCMC\n15319689\n3/25/1998 12:00:00...,SMOKER,SMOKER
2,\n071962960\nBH\n4236518\n417454\n12/10/2001 1...,SMOKER,SMOKER
3,\n418520250\nNVH\n61562872\n3/11/1995 12:00:00...,SMOKER,SMOKER
4,\n301443520\nCTMC\n49020928\n448922\n1/11/1990...,SMOKER,SMOKER


In [37]:
df_debug.Predicted.value_counts()

SMOKER    146
Name: Predicted, dtype: int64